In [139]:
pip install -r requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.1.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [140]:
from llama_index.readers.obsidian import ObsidianReader
from llama_index.core.memory.chat_memory_buffer import MessageRole
from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex, VectorStoreIndex
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import Document, PropertyGraphIndex
from llama_index.core.storage.index_store import SimpleIndexStore
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.core import Settings
from IPython.display import Markdown, display
from llama_index.llms.ollama import Ollama
from tqdm.notebook import tqdm
import time
import os
from llama_index.core.llms import ChatMessage
from llama_index.core import StorageContext
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.memory import ChatMemoryBuffer
import logging
import sys
import ipywidgets as widgets
import json
from llama_index.core.callbacks import CallbackManager
from llama_index.core.callbacks import LlamaDebugHandler
from llama_index.core import ServiceContext
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever
from llama_index.core.indices.property_graph import (
    SimpleLLMPathExtractor,
    SchemaLLMPathExtractor,
    DynamicLLMPathExtractor,
)
import yaml
import networkx as nx
from pyvis.network import Network


In [141]:
from llama_index.core import (
    load_index_from_storage,
    load_indices_from_storage,
    load_graph_from_storage,
)

In [142]:
import nest_asyncio

nest_asyncio.apply()

In [143]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Set LLM (OpenAI)

In [144]:
load_dotenv()  # Charge les variables depuis le fichier .env
api_key = os.getenv("OPENAI_API_KEY_UP")
# Modifier ou ajouter une variable d'environnement
os.environ["OPENAI_API_KEY"] = api_key

In [145]:
llm = OpenAI(temperature=0, model="gpt-4o", max_tokens=3000)
Settings.llm = llm
Settings.chunk_size = 512

# Set local LLM for embeddings

In [146]:
# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
#Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


# Set LLM for chat  (Local)

In [147]:
#llm = Ollama(model="tinyllama", request_timeout=120.0)
#Settings.llm = llm
#Settings.chunk_size = 512

# Test LLM

In [148]:
messages = [
    ChatMessage(
        role="system", content="You are a data governance consultant"
    ),
    ChatMessage(role="user", content="What's your favorite data tool ?"),
]
resp = llm.chat(messages)
print(resp)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
assistant: As a data governance consultant, I don't have personal preferences or feelings. However, I can provide insights into popular data tools that are widely used in the industry for data governance purposes. Some of these include:

1. **Collibra**: Known for its comprehensive data governance platform, Collibra helps organizations manage data assets, ensure data quality, and maintain compliance with regulations.

2. **Informatica**: Offers a suite of data governance tools that include data cataloging, data quality, and master data management, helping organizations to streamline

# Load storage contexts

## Load vector storage context

In [149]:
vector_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="vector"),
    vector_store=SimpleVectorStore.from_persist_dir(
        persist_dir="vector"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="vector"),
)

## Load knowledge graph storage context

In [150]:
graph_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="knowledge_graph"),
    graph_store=SimpleGraphStore.from_persist_dir(
        persist_dir="knowledge_graph"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="knowledge_graph"),
)

## Load onto graph storage context


In [151]:
onto_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="onto_graph"),
    graph_store=SimpleGraphStore.from_persist_dir(
        persist_dir="onto_graph"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="onto_graph"),
)

In [152]:
print(onto_storage_context)

StorageContext(docstore=<llama_index.core.storage.docstore.simple_docstore.SimpleDocumentStore object at 0x2c678df90>, index_store=<llama_index.core.storage.index_store.simple_index_store.SimpleIndexStore object at 0x2a2ae4810>, vector_stores={'default': SimpleVectorStore(stores_text=False, is_embedding_query=True, data=SimpleVectorStoreData(embedding_dict={}, text_id_to_ref_doc_id={}, metadata_dict={})), 'image': SimpleVectorStore(stores_text=False, is_embedding_query=True, data=SimpleVectorStoreData(embedding_dict={}, text_id_to_ref_doc_id={}, metadata_dict={}))}, graph_store=<llama_index.core.graph_stores.simple.SimpleGraphStore object at 0x2c0bbd910>, property_graph_store=None)


# Load index

## Load vector index

In [153]:
simple_index = load_index_from_storage(vector_storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.


### Set retriever

In [154]:
simple_query_engine = simple_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

### Test retriever

In [155]:
simple_rag_retriever = simple_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True
)

response = simple_query_engine.query(
    "Qui est-tu ?",
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [156]:
display(Markdown(f"<b>{response}</b>"))

<b>Je suis un système de questions-réponses conçu pour fournir des réponses précises et fiables en utilisant les informations disponibles.</b>

## Load graph index

In [157]:
graph_index = load_index_from_storage(graph_storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.


In [158]:
nx_graph = graph_index.get_networkx_graph()

In [159]:
# Count the number of nodes
num_nodes = len(nx_graph.edges())

print(f"Number of edges in the knowledge graph: {num_nodes}")

Number of edges in the knowledge graph: 60


In [160]:
g = graph_index.get_networkx_graph()
print(g)
print("Nodes:", g.nodes())
print("Edges:", g.edges())
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)

with open("knowledge_graph.html", "w", encoding="utf-8") as f:
    f.write(net.generate_html())

Graph with 94 nodes and 60 edges
Nodes: ['Exactitude', 'Justes et cohérentes', 'Complétude', 'Complet', 'Nettoyage', 'Erreurs', 'Normalisation', 'Formats', 'Florence nightingale', 'Infirmière britannique', "Présentation visuelle de l'information", 'Dataviz project', 'Relation', 'Bibliothèque', 'Description', 'Visualisation de données', 'Structure de données', 'Volume', 'Quantité massive de données', 'Vélocité', 'Vitesse de génération de données', 'Système', 'Stocker gérer et récupérer des données', 'Base de données relationnelles', 'Relations entre les tables', 'Tirer', 'Données', 'Exploitables', 'Format prédéfini', 'Texte libre', 'Caractéristiques', 'Claire et standardisée', 'Passer', 'Interprétation des tendances', 'Data visualisation', 'Données brutes', 'Information claire', 'Processus', 'Visualisation', 'Objectif', 'Experts', 'Dataviz', 'Page layout', 'Interactive data visualization', 'Edward rolf tufte', 'Professor', 'Léonard de vinci des données', 'Théophile', 'Outil informatique

### Set retriever

In [161]:
graph_query_engine = graph_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

### Test retriever

In [162]:
graph_rag_retriever = graph_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True
)

response = graph_query_engine.query(
    "Qui est-tu ?",
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 8140ac08-3cb9-4e54-a6b5-1e8e8ce3c81d: Données qui n'ont pas de format prédéfini ou organisé. Elles peuvent contenir...
> Querying with idx: 8140ac08-3cb9-4e54-a6b5-1e8e8ce3c81d: Données qui n'ont pas de format prédéfini ou organisé. Elles peuvent contenir...
> Querying with idx: 8140ac08-3cb9-4e54-a6b5-1e8e8ce3c81d: Données qui n'ont pas de format prédéfini ou organisé. Elles peuvent contenir...
> Querying with idx: 8140ac08-3cb9-4e54-a6b5-1e8e8ce3c81d: Données qui n'ont pas de format prédéfini ou organisé. Elles peuvent contenir...
> Querying with idx: 8140ac08-3cb9-4e54-a6b5-1e8e8ce3c81d: Données qui n'ont pas de format prédéfini ou organisé. Elles peuvent contenir...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 8dfe6545-46a0-4a59-ac7d-2d5d45df7ed0: Quels critères pour dire que des données sont de qualité ?
- Exactitude : les...
> Querying with idx: 8dfe6545-46a0-4a59-ac7d-2

In [163]:
display(Markdown(f"<b>{response}</b>"))

<b>Je suis un système de questions-réponses conçu pour fournir des réponses précises et fiables en utilisant les informations disponibles.</b>

## Load onto index

In [164]:
onto_storage_context = StorageContext.from_defaults(persist_dir="onto_graph")
# Load the PropertyGraphIndex from the storage context

onto_index = load_index_from_storage(onto_storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.


In [165]:
onto_index.property_graph_store.save_networkx_graph(
    name="OntoGraph.html"
)

### Set retriever

In [166]:
onto_query_engine = onto_index.as_query_engine(
 include_text=True,
 similarity_top_k=8,
)

### Test retriever

In [167]:
onto_rag_retriever = onto_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True
)

response = onto_query_engine.query(
    "Qui est-tu ?",
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [168]:
display(Markdown(f"<b>{response}</b>"))

<b>Je suis un système de questions-réponses conçu pour fournir des réponses précises et fiables en utilisant les informations disponibles.</b>

# Visualize knowledge graph

In [169]:
nx_graph = graph_index.get_networkx_graph()

In [170]:
# Count the number of nodes
num_nodes = len(nx_graph.edges())

print(f"Number of nodes in the knowledge graph: {num_nodes}")

Number of nodes in the knowledge graph: 60


In [171]:
g = graph_index.get_networkx_graph()
print(g)
print("Nodes:", g.nodes())
print("Edges:", g.edges())
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)

with open("knowledge_graph.html", "w", encoding="utf-8") as f:
    f.write(net.generate_html())

Graph with 94 nodes and 60 edges
Nodes: ['Exactitude', 'Justes et cohérentes', 'Complétude', 'Complet', 'Nettoyage', 'Erreurs', 'Normalisation', 'Formats', 'Florence nightingale', 'Infirmière britannique', "Présentation visuelle de l'information", 'Dataviz project', 'Relation', 'Bibliothèque', 'Description', 'Visualisation de données', 'Structure de données', 'Volume', 'Quantité massive de données', 'Vélocité', 'Vitesse de génération de données', 'Système', 'Stocker gérer et récupérer des données', 'Base de données relationnelles', 'Relations entre les tables', 'Tirer', 'Données', 'Exploitables', 'Format prédéfini', 'Texte libre', 'Caractéristiques', 'Claire et standardisée', 'Passer', 'Interprétation des tendances', 'Data visualisation', 'Données brutes', 'Information claire', 'Processus', 'Visualisation', 'Objectif', 'Experts', 'Dataviz', 'Page layout', 'Interactive data visualization', 'Edward rolf tufte', 'Professor', 'Léonard de vinci des données', 'Théophile', 'Outil informatique

# (Simple) Query the vectors

In [172]:
query = "Qui est-tu ?"
query_engine = simple_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

response = query_engine.query(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [173]:
display(Markdown(f"<b>{response}</b>"))

<b>Je suis un système de questions-réponses conçu pour fournir des réponses précises et fiables en utilisant les informations disponibles. Mon objectif est d'aider à clarifier des sujets en fonction des données fournies.</b>

# (Simple) Query the knowledge graph 

In [174]:
query = "Qui est-tu ?"
graph_query_engine = graph_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

response = graph_query_engine.query(query)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 8140ac08-3cb9-4e54-a6b5-1e8e8ce3c81d: Données qui n'ont pas de format prédéfini ou organisé. Elles peuvent contenir...
> Querying with idx: 8140ac08-3cb9-4e54-a6b5-1e8e8ce3c81d: Données qui n'ont pas de format prédéfini ou organisé. Elles peuvent contenir...
> Querying with idx: 8140ac08-3cb9-4e54-a6b5-1e8e8ce3c81d: Données qui n'ont pas de format prédéfini ou organisé. Elles peuvent contenir...
> Querying with idx: 8140ac08-3cb9-4e54-a6b5-1e8e8ce3c81d: Données qui n'ont pas de format prédéfini ou organisé. Elles peuvent contenir...
> Querying with idx: 8140ac08-3cb9-4e54-a6b5-1e8e8ce3c81d: Données qui n'ont pas de format prédéfini ou organisé. Elles peuvent contenir...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 8dfe6545-46a0-4a59-ac7d-2d5d45df7ed0: Quels critères pour dire que des données sont de qualité ?
- Exactitude : les...
> Querying with idx: 8dfe6545-46a0-4a59-ac7d-2

In [175]:
display(Markdown(f"<b>{response}</b>"))

<b>Je suis un système de questions-réponses conçu pour fournir des réponses précises et fiables en utilisant les informations disponibles.</b>

# (Simple) Query the onto graph 

In [176]:
query = "Qui est-tu ?"
onto_query_engine = onto_index.as_query_engine(
 include_text=True,
 similarity_top_k=2,
)

response = onto_query_engine.query(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [177]:
display(Markdown(f"<b>{response}</b>"))

<b>Je suis un système de questions-réponses conçu pour fournir des réponses précises en utilisant les informations disponibles.</b>

## (Node retriever)

In [178]:
retriever = onto_index.as_retriever(
    include_text=True,  # include source text, default True
)

nodes = retriever.retrieve("Qui est-tu ?")


for node in nodes:
    print(node.text)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Here are some facts extracted from the provided text:

dataviz -> INVOLVES -> interaction entre l'homme et la donnée

Tout n'est que données 
L'art de la [[dataviz]] c'est de transformer un tas de trucs qu'on sait pas trop comment définir en des connaissances et des idées et leur donner une application dans la vie courante.

**Exemple :** Le service communication d'une entreprise analyse les données relatives à l'envoi d'une newsletter pour comprendre ce qui a mieux ou moins bien marché. Ils utilisent des outils qui mettent en forment les données et en produisent des évaluations, pa

# Have a real chat with your data

## Set up the engines

### Vector engines

In [179]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
vector_chat_engine = simple_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
        " "
        " "
        "."
    ),
    verbose=False,
)

### Graph engines

In [180]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
graph_chat_engine = graph_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    verbose=False,
)

### Onto engines

In [181]:
#onto_chat_engine = onto_index.query_engine(
#    chat_mode="condense_plus_context",
#    llm=llm
#)

## Generate

In [182]:
graph_chat_engine.reset()

In [183]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
graph_chat_engine = graph_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
        "Tu dois générer des générer des méthodologies de dataviz centrées autour de la découverte de lignes de force entre idées"
        " "
        " "
    ),
    verbose=False,
)

In [184]:
response_stream = graph_chat_engine.stream_chat("""D'après ta base de connaissance et ta vision du monde, dans le cadre d'une analyse de l'impact des tendances économiques sur les comportements de consommation, créer un diagramme de corrélation entre les idées principales telles que la croissance économique, l'inflation, et les choix des consommateurs, pour visualiser les relations fortes et faibles entre ces variables.""")

INFO:llama_index.core.chat_engine.condense_plus_context:Condensed question: D'après ta base de connaissance et ta vision du monde, dans le cadre d'une analyse de l'impact des tendances économiques sur les comportements de consommation, créer un diagramme de corrélation entre les idées principales telles que la croissance économique, l'inflation, et les choix des consommateurs, pour visualiser les relations fortes et faibles entre ces variables.
Condensed question: D'après ta base de connaissance et ta vision du monde, dans le cadre d'une analyse de l'impact des tendances économiques sur les comportements de consommation, créer un diagramme de corrélation entre les idées principales telles que la croissance économique, l'inflation, et les choix des consommateurs, pour visualiser les relations fortes et faibles entre ces variables.
Condensed question: D'après ta base de connaissance et ta vision du monde, dans le cadre d'une analyse de l'impact des tendances économiques sur les comportem

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 13c12931-e4e2-4bc8-b819-40e2ffae0d7f: En fonction de la nature des données, on ne choisit pas les mêmes types de re...
> Querying with idx: 13c12931-e4e2-4bc8-b819-40e2ffae0d7f: En fonction de la nature des données, on ne choisit pas les mêmes types de re...
> Querying with idx: 13c12931-e4e2-4bc8-b819-40e2ffae0d7f: En fonction de la nature des données, on ne choisit pas les mêmes types de re...
> Querying with idx: 13c12931-e4e2-4bc8-b819-40e2ffae0d7f: En fonction de la nature des données, on ne choisit pas les mêmes types de re...
> Querying with idx: 13c12931-e4e2-4bc8-b819-40e2ffae0d7f: En fonction de la nature des données, on ne choisit pas les mêmes types de re...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: be5f0df9-4609-4a4a-b429-e3529e134898: ---
partie de:
  - "[[Dataviz Project]]"
  - "[[dataviz]]"
---
A chaque visua...
> Querying with idx: be5f0df9-4609-4a4a-b429-e

In [185]:
generate = response_stream.print_response_stream()


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Pour créer un diagramme de corrélation qui visualise les relations entre la croissance économique, l'inflation, et les choix des consommateurs, nous pouvons suivre une méthodologie structurée. Voici les étapes pour réaliser cette datavisualisation :

### Étape 1 : Collecte de données
- **Sources de données** : Rassemblez des données historiques sur la croissance économique (PIB), l'inflation (IPC), et les comportements de consommation (dépenses des ménages, préférences d'achat, etc.) à partir de sources fiables comme les banques centrales, les instituts de statistiques nationaux, ou

In [186]:
display(Markdown(f"<b>{generate}</b>"))

<b>None</b>

In [187]:
section_modele = """ 


"""

In [188]:
response_stream = graph_chat_engine.stream_chat("""D'après ta base de connaissance et ta vision du monde, dans le cadre d'une recherche sur l'innovation technologique, concevoir un graphique de type réseau où les nœuds représentent des idées comme la R&D, la disruption technologique, et l'adoption par les consommateurs, et les arêtes indiquent les connexions les plus solides et significatives. Imite le style d'écriture de la {section_modele}""")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.chat_engine.condense_plus_context:Condensed question: Dans le cadre d'une recherche sur l'innovation technologique, comment concevoir un graphique de type réseau où les nœuds représentent des idées telles que la R&D, la disruption technologique, et l'adoption par les consommateurs, et où les arêtes indiquent les connexions les plus solides et significatives, en imitant le style d'écriture de la section modèle ?
Condensed question: Dans le cadre d'une recherche sur l'innovation technologique, comment concevoir un graphique de type réseau où les nœuds représenten

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 2bae5942-7afe-48d2-a354-ae38118d8fea: Manière dont les utilisateurs interprètent les données au travers de la visua...
> Querying with idx: 2bae5942-7afe-48d2-a354-ae38118d8fea: Manière dont les utilisateurs interprètent les données au travers de la visua...
> Querying with idx: 2bae5942-7afe-48d2-a354-ae38118d8fea: Manière dont les utilisateurs interprètent les données au travers de la visua...
> Querying with idx: 2bae5942-7afe-48d2-a354-ae38118d8fea: Manière dont les utilisateurs interprètent les données au travers de la visua...
> Querying with idx: 2bae5942-7afe-48d2-a354-ae38118d8fea: Manière dont les utilisateurs interprètent les données au travers de la visua...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 3a4d198b-ec6a-4de4-988b-a1bf59971291: Tout n'est que données 
L'art de la [[dataviz]] c'est de transformer un tas d...
> Querying with idx: 3a4d198b-ec6a-4de4-988b-a

In [189]:
generate = response_stream.print_response_stream()


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Pour concevoir un graphique de type réseau dans le cadre d'une recherche sur l'innovation technologique, nous allons suivre une méthodologie qui met en avant les connexions significatives entre des idées clés telles que la R&D, la disruption technologique, et l'adoption par les consommateurs. Voici comment procéder :

### Étape 1 : Identification des idées clés
- **Nœuds principaux** : Définissez les idées principales qui seront représentées comme des nœuds dans le réseau. Dans ce cas, les nœuds incluront la Recherche et Développement (R&D), la disruption technologique, l'adoption p

In [190]:
display(Markdown(f"<b>{generate}</b>"))

<b>None</b>

## Sum-up

In [191]:
chat_history = memory.get_all()

# Assuming chat_history is available and contains your messages
assistant_messages = [
    message.content 
    for message in chat_history 
    if message.role == MessageRole.ASSISTANT  # Compare with the enum directly
]


output_filename = r"/Users/arthursarazin/Documents/coreandgraphs/graphandvisualize/output.md"
# Write to a Markdown file
with open(output_filename, "w", encoding="utf-8") as f:
    for msg in assistant_messages:
        f.write(msg + "\n\n") 